In [7]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import math
import pyautogui
import time
import mss.tools
import uuid
import keyboard
from pynput import mouse


In [3]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("./converted_keras/keras_Model.h5", compile=False)

# Load the labels
class_names = open("./converted_keras/labels.txt", "r").readlines()

In [37]:
def screenshot():
    with mss.mss() as sct:
        time.sleep(1)
        monitor_info = sct.monitors[0]
        top_margin = (monitor_info["height"] // 4) + 50
        monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-450}
        output = f"madness/{str(uuid.uuid4())}.png".format(**monitor)
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    return output  

In [ ]:
def predict_direction(image):
    # Preprocess the image
    image = cv2.resize(image, (270, 1800))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = np.reshape(image, (1, -1))
    image = np.expand_dims(image, axis=0)
 
    # Predict the direction
    #direction = model.predict(image)[1][0][0]
    direction = model.predict(image)
    return direction

In [39]:
def predict_direction():
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    path = screenshot()
    image = Image.open(path).convert("RGB")
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    class_name = class_name[:1]
    return class_name
    


In [ ]:
time.sleep(5)
while not keyboard.is_pressed("$"):
    predicted_class = predict_direction()
    print(f"Predicted: {predict_direction()}")
    #if confidence < threshold:
    if (predicted_class == '0'):
        keyboard.press('d')
        pyautogui.mouseDown()
        time.sleep(0.25)
        pyautogui.mouseUp()
        keyboard.release('d')
    elif (predicted_class == '2'):
        keyboard.press('q')
        pyautogui.mouseDown()
        time.sleep(0.25)
        pyautogui.mouseUp()
        keyboard.release('q')
    else:
        pyautogui.mouseDown()
        time.sleep(0.3)           
        pyautogui.mouseUp()

In [22]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

path = "./data_deep_learning_iris_v2/forward/014e986b-da10-43c4-8cce-2377a0adf2ce.png"
# Replace this with the path to your image
image = Image.open(path).convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

1/1 [==============================] - 0s 31ms/step
Class: Forward
Confidence Score: 0.99996734
